# Ingeniería de datos

## Carga de modulos

In [ ]:
# Data wrangling
import json as js
import numpy as np
import pandas as pd
from sqlalchemy import create_engine

# Data viz
import cufflinks as cf

# Enviroment setup
cf.go_offline()
pd.set_option('display.max_columns', 50)
pd.set_option('display.float_format', lambda x: '%.2f' % x)

## Carga de datos

In [ ]:
crdcard_b = pd.read_csv('/content/drive/MyDrive/datos/credit_card_balance.csv')
inst_paym = pd.read_csv('/content/drive/MyDrive/datos/installments_payments.csv')
poscash_b = pd.read_csv('/content/drive/MyDrive/datos/POS_CASH_balance.csv')
prev_app = pd.read_csv('/content/drive/MyDrive/datos/previous_application.csv')
bureau_b = pd.read_csv('/content/drive/MyDrive/datos/bureau_balance.csv')
bureau = pd.read_csv('/content/drive/MyDrive/datos/bureau.csv')

In [ ]:
descrip = pd.read_csv('/content/drive/MyDrive/datos/HomeCredit_columns_description.csv', encoding= 'unicode_escape')
#app_test = pd.read_csv('/content/drive/MyDrive/datos/application_test.csv')
#app_train = pd.read_csv('/content/drive/MyDrive/datos/application_train.csv')
#sample_sb = pd.read_csv('/content/drive/MyDrive/datos/sample_submission.csv')

## Funciones auxiliares

In [ ]:
def descrip_column(table):
  with pd.option_context('display.max_colwidth', None):
    display(descrip[descrip['Table']==table+'.csv'].drop(columns=descrip.columns[0:2]))

In [ ]:
def input_missing(table, coldisc, colcont):
  for feature, missing in list(table.isna().mean().items()):
    if missing > 0 and missing < 0.35:
      if feature in colcont:
        table[feature]=table[feature].fillna(table[feature].mean())
      elif feature in coldisc:
        table[feature]=table[feature].fillna(table[feature].mode())
    elif missing > 0.35:
      del table[feature]
      if feature in colcont:
        del colcont[colcont.index(feature)]
      elif feature in coldisc:
        del coldisc[coldisc.index(feature)]
  display(table.isna().mean())

In [ ]:
def get_tabledummies(table, coldisc):
  for feature in coldisc:
    tmp = pd.get_dummies(table[feature])
    for dum in tmp.columns:
      if str(dum).replace('.','',1).isdigit():
        table[feature+'_'+str(int(dum))]=tmp[dum]
      else:
        table[feature+'_'+str(dum).upper()]=tmp[dum]
    del table[feature]
  display(table)

## Limpieza de Datos

### Tabla credit_card_balance

#### Imputación

In [ ]:
ls_crdcard_disc = ['NAME_CONTRACT_STATUS']
ls_crdcard_cont = list(crdcard_b.columns.drop(ls_crdcard_disc))

In [ ]:
crdcard_b.isna().mean()

SK_ID_PREV                   0.00
SK_ID_CURR                   0.00
MONTHS_BALANCE               0.00
AMT_BALANCE                  0.00
AMT_CREDIT_LIMIT_ACTUAL      0.00
AMT_DRAWINGS_ATM_CURRENT     0.20
AMT_DRAWINGS_CURRENT         0.00
AMT_DRAWINGS_OTHER_CURRENT   0.20
AMT_DRAWINGS_POS_CURRENT     0.20
AMT_INST_MIN_REGULARITY      0.08
AMT_PAYMENT_CURRENT          0.20
AMT_PAYMENT_TOTAL_CURRENT    0.00
AMT_RECEIVABLE_PRINCIPAL     0.00
AMT_RECIVABLE                0.00
AMT_TOTAL_RECEIVABLE         0.00
CNT_DRAWINGS_ATM_CURRENT     0.20
CNT_DRAWINGS_CURRENT         0.00
CNT_DRAWINGS_OTHER_CURRENT   0.20
CNT_DRAWINGS_POS_CURRENT     0.20
CNT_INSTALMENT_MATURE_CUM    0.08
NAME_CONTRACT_STATUS         0.00
SK_DPD                       0.00
SK_DPD_DEF                   0.00
dtype: float64

In [ ]:
input_missing(crdcard_b, ls_crdcard_disc, ls_crdcard_cont)

SK_ID_PREV                   0.00
SK_ID_CURR                   0.00
MONTHS_BALANCE               0.00
AMT_BALANCE                  0.00
AMT_CREDIT_LIMIT_ACTUAL      0.00
AMT_DRAWINGS_ATM_CURRENT     0.00
AMT_DRAWINGS_CURRENT         0.00
AMT_DRAWINGS_OTHER_CURRENT   0.00
AMT_DRAWINGS_POS_CURRENT     0.00
AMT_INST_MIN_REGULARITY      0.00
AMT_PAYMENT_CURRENT          0.00
AMT_PAYMENT_TOTAL_CURRENT    0.00
AMT_RECEIVABLE_PRINCIPAL     0.00
AMT_RECIVABLE                0.00
AMT_TOTAL_RECEIVABLE         0.00
CNT_DRAWINGS_ATM_CURRENT     0.00
CNT_DRAWINGS_CURRENT         0.00
CNT_DRAWINGS_OTHER_CURRENT   0.00
CNT_DRAWINGS_POS_CURRENT     0.00
CNT_INSTALMENT_MATURE_CUM    0.00
NAME_CONTRACT_STATUS         0.00
SK_DPD                       0.00
SK_DPD_DEF                   0.00
dtype: float64

#### Obtener dummies discretas




In [ ]:
crdcard_b

,SK_ID_PREV,SK_ID_CURR,MONTHS_BALANCE,AMT_BALANCE,AMT_CREDIT_LIMIT_ACTUAL,AMT_DRAWINGS_ATM_CURRENT,AMT_DRAWINGS_CURRENT,AMT_DRAWINGS_OTHER_CURRENT,AMT_DRAWINGS_POS_CURRENT,AMT_INST_MIN_REGULARITY,AMT_PAYMENT_CURRENT,AMT_PAYMENT_TOTAL_CURRENT,AMT_RECEIVABLE_PRINCIPAL,AMT_RECIVABLE,AMT_TOTAL_RECEIVABLE,CNT_DRAWINGS_ATM_CURRENT,CNT_DRAWINGS_CURRENT,CNT_DRAWINGS_OTHER_CURRENT,CNT_DRAWINGS_POS_CURRENT,CNT_INSTALMENT_MATURE_CUM,NAME_CONTRACT_STATUS,SK_DPD,SK_DPD_DEF
0,2562384,378907,-6,56.97,135000,0.00,877.50,0.00,877.50,1700.33,1800.00,1800.00,0.00,0.00,0.00,0.00,1,0.00,1.00,35.00,Active,0,0
1,2582071,363914,-1,63975.56,45000,2250.00,2250.00,0.00,0.00,2250.00,2250.00,2250.00,60175.08,64875.56,64875.56,1.00,1,0.00,0.00,69.00,Active,0,0
2,1740877,371185,-7,31815.22,450000,0.00,0.00,0.00,0.00,2250.00,2250.00,2250.00,26926.42,31460.08,31460.08,0.00,0,0.00,0.00,30.00,Active,0,0
3,1389973,337855,-4,236572.11,225000,2250.00,2250.00,0.00,0.00,11795.76,11925.00,11925.00,224949.29,233048.97,233048.97,1.00,1,0.00,0.00,10.00,Active,0,0
4,1891521,126868,-1,453919.46,450000,0.00,11547.00,0.00,11547.00,22924.89,27000.00,27000.00,443044.40,453919.46,453919.46,0.00,1,0.00,1.00,101.00,Active,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3840307,1036507,328243,-9,0.00,45000,5961.32,0.00,288.17,2968.80,0.00,10280.54,0.00,0.00,0.00,0.00,0.31,0,0.00,0.56,0.00,Active,0,0
3840308,1714892,347207,-9,0.00,45000,0.00,0.00,0.00,0.00,0.00,1879.11,0.00,0.00,0.00,0.00,0.00,0,0.00,0.00,23.00,Active,0,0
3840309,1302323,215757,-9,275784.97,585000,270000.00,270000.00,0.00,0.00,2250.00,375750.00,356994.67,269356.14,273093.97,273093.97,2.00,2,0.00,0.00,18.00,Active,0,0
3840310,1624872,430337,-10,0.00,450000,5961.32,0.00,288.17,2968.80,0.00,10280.54,0.00,0.00,0.00,0.00,0.31,0,0.00,0.56,0.00,Active,0,0


In [ ]:
get_tabledummies(crdcard_b,ls_crdcard_disc)

,SK_ID_PREV,SK_ID_CURR,MONTHS_BALANCE,AMT_BALANCE,AMT_CREDIT_LIMIT_ACTUAL,AMT_DRAWINGS_ATM_CURRENT,AMT_DRAWINGS_CURRENT,AMT_DRAWINGS_OTHER_CURRENT,AMT_DRAWINGS_POS_CURRENT,AMT_INST_MIN_REGULARITY,AMT_PAYMENT_CURRENT,AMT_PAYMENT_TOTAL_CURRENT,AMT_RECEIVABLE_PRINCIPAL,AMT_RECIVABLE,AMT_TOTAL_RECEIVABLE,CNT_DRAWINGS_ATM_CURRENT,CNT_DRAWINGS_CURRENT,CNT_DRAWINGS_OTHER_CURRENT,CNT_DRAWINGS_POS_CURRENT,CNT_INSTALMENT_MATURE_CUM,SK_DPD,SK_DPD_DEF,NAME_CONTRACT_STATUS_ACTIVE,NAME_CONTRACT_STATUS_APPROVED,NAME_CONTRACT_STATUS_COMPLETED,NAME_CONTRACT_STATUS_DEMAND,NAME_CONTRACT_STATUS_REFUSED,NAME_CONTRACT_STATUS_SENT PROPOSAL,NAME_CONTRACT_STATUS_SIGNED
0,2562384,378907,-6,56.97,135000,0.00,877.50,0.00,877.50,1700.33,1800.00,1800.00,0.00,0.00,0.00,0.00,1,0.00,1.00,35.00,0,0,1,0,0,0,0,0,0
1,2582071,363914,-1,63975.56,45000,2250.00,2250.00,0.00,0.00,2250.00,2250.00,2250.00,60175.08,64875.56,64875.56,1.00,1,0.00,0.00,69.00,0,0,1,0,0,0,0,0,0
2,1740877,371185,-7,31815.22,450000,0.00,0.00,0.00,0.00,2250.00,2250.00,2250.00,26926.42,31460.08,31460.08,0.00,0,0.00,0.00,30.00,0,0,1,0,0,0,0,0,0
3,1389973,337855,-4,236572.11,225000,2250.00,2250.00,0.00,0.00,11795.76,11925.00,11925.00,224949.29,233048.97,233048.97,1.00,1,0.00,0.00,10.00,0,0,1,0,0,0,0,0,0
4,1891521,126868,-1,453919.46,450000,0.00,11547.00,0.00,11547.00,22924.89,27000.00,27000.00,443044.40,453919.46,453919.46,0.00,1,0.00,1.00,101.00,0,0,1,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3840307,1036507,328243,-9,0.00,45000,5961.32,0.00,288.17,2968.80,0.00,10280.54,0.00,0.00,0.00,0.00,0.31,0,0.00,0.56,0.00,0,0,1,0,0,0,0,0,0
3840308,1714892,347207,-9,0.00,45000,0.00,0.00,0.00,0.00,0.00,1879.11,0.00,0.00,0.00,0.00,0.00,0,0.00,0.00,23.00,0,0,1,0,0,0,0,0,0
3840309,1302323,215757,-9,275784.97,585000,270000.00,270000.00,0.00,0.00,2250.00,375750.00,356994.67,269356.14,273093.97,273093.97,2.00,2,0.00,0.00,18.00,0,0,1,0,0,0,0,0,0
3840310,1624872,430337,-10,0.00,450000,5961.32,0.00,288.17,2968.80,0.00,10280.54,0.00,0.00,0.00,0.00,0.31,0,0.00,0.56,0.00,0,0,1,0,0,0,0,0,0



### Tabla installment_payments



#### Imputación

In [ ]:
ls_inspay_disc = ['NUM_INSTALMENT_VERSION']
ls_inspay_cont = list(inst_paym.columns.drop(ls_inspay_disc))

In [ ]:
inst_paym.isna().mean()

SK_ID_PREV               0.00
SK_ID_CURR               0.00
NUM_INSTALMENT_VERSION   0.00
NUM_INSTALMENT_NUMBER    0.00
DAYS_INSTALMENT          0.00
DAYS_ENTRY_PAYMENT       0.00
AMT_INSTALMENT           0.00
AMT_PAYMENT              0.00
dtype: float64

In [ ]:
input_missing(inst_paym, ls_inspay_disc, ls_inspay_cont)

SK_ID_PREV               0.00
SK_ID_CURR               0.00
NUM_INSTALMENT_VERSION   0.00
NUM_INSTALMENT_NUMBER    0.00
DAYS_INSTALMENT          0.00
DAYS_ENTRY_PAYMENT       0.00
AMT_INSTALMENT           0.00
AMT_PAYMENT              0.00
dtype: float64

#### Obtener dummies discretas

In [ ]:
inst_paym

,SK_ID_PREV,SK_ID_CURR,NUM_INSTALMENT_VERSION,NUM_INSTALMENT_NUMBER,DAYS_INSTALMENT,DAYS_ENTRY_PAYMENT,AMT_INSTALMENT,AMT_PAYMENT
0,1054186,161674,1.00,6,-1180.00,-1187.00,6948.36,6948.36
1,1330831,151639,0.00,34,-2156.00,-2156.00,1716.53,1716.53
2,2085231,193053,2.00,1,-63.00,-63.00,25425.00,25425.00
3,2452527,199697,1.00,3,-2418.00,-2426.00,24350.13,24350.13
4,2714724,167756,1.00,2,-1383.00,-1366.00,2165.04,2160.59
...,...,...,...,...,...,...,...,...
13605396,2186857,428057,0.00,66,-1624.00,-1051.11,67.50,17238.22
13605397,1310347,414406,0.00,47,-1539.00,-1051.11,67.50,17238.22
13605398,1308766,402199,0.00,43,-7.00,-1051.11,43737.43,17238.22
13605399,1062206,409297,0.00,43,-1986.00,-1051.11,67.50,17238.22


In [ ]:
get_tabledummies(inst_paym, ls_inspay_disc)

,SK_ID_PREV,SK_ID_CURR,NUM_INSTALMENT_NUMBER,DAYS_INSTALMENT,DAYS_ENTRY_PAYMENT,AMT_INSTALMENT,AMT_PAYMENT,NUM_INSTALMENT_VERSION_0,NUM_INSTALMENT_VERSION_1,NUM_INSTALMENT_VERSION_2,NUM_INSTALMENT_VERSION_3,NUM_INSTALMENT_VERSION_4,NUM_INSTALMENT_VERSION_5,NUM_INSTALMENT_VERSION_6,NUM_INSTALMENT_VERSION_7,NUM_INSTALMENT_VERSION_8,NUM_INSTALMENT_VERSION_9,NUM_INSTALMENT_VERSION_10,NUM_INSTALMENT_VERSION_11,NUM_INSTALMENT_VERSION_12,NUM_INSTALMENT_VERSION_13,NUM_INSTALMENT_VERSION_14,NUM_INSTALMENT_VERSION_15,NUM_INSTALMENT_VERSION_16,NUM_INSTALMENT_VERSION_17,...,NUM_INSTALMENT_VERSION_40,NUM_INSTALMENT_VERSION_41,NUM_INSTALMENT_VERSION_42,NUM_INSTALMENT_VERSION_43,NUM_INSTALMENT_VERSION_44,NUM_INSTALMENT_VERSION_45,NUM_INSTALMENT_VERSION_46,NUM_INSTALMENT_VERSION_47,NUM_INSTALMENT_VERSION_48,NUM_INSTALMENT_VERSION_49,NUM_INSTALMENT_VERSION_50,NUM_INSTALMENT_VERSION_51,NUM_INSTALMENT_VERSION_52,NUM_INSTALMENT_VERSION_53,NUM_INSTALMENT_VERSION_54,NUM_INSTALMENT_VERSION_55,NUM_INSTALMENT_VERSION_56,NUM_INSTALMENT_VERSION_57,NUM_INSTALMENT_VERSION_58,NUM_INSTALMENT_VERSION_59,NUM_INSTALMENT_VERSION_61,NUM_INSTALMENT_VERSION_68,NUM_INSTALMENT_VERSION_72,NUM_INSTALMENT_VERSION_73,NUM_INSTALMENT_VERSION_178
0,1054186,161674,6,-1180.00,-1187.00,6948.36,6948.36,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,1330831,151639,34,-2156.00,-2156.00,1716.53,1716.53,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,2085231,193053,1,-63.00,-63.00,25425.00,25425.00,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,2452527,199697,3,-2418.00,-2426.00,24350.13,24350.13,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,2714724,167756,2,-1383.00,-1366.00,2165.04,2160.59,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
13605396,2186857,428057,66,-1624.00,-1051.11,67.50,17238.22,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
13605397,1310347,414406,47,-1539.00,-1051.11,67.50,17238.22,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
13605398,1308766,402199,43,-7.00,-1051.11,43737.43,17238.22,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
13605399,1062206,409297,43,-1986.00,-1051.11,67.50,17238.22,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


### Tabla bureau


#### Imputación

In [ ]:
ls_bureau_disc = ['CREDIT_ACTIVE','CREDIT_CURRENCY','CREDIT_TYPE'] 
ls_bureau_cont = list(bureau.columns.drop(ls_bureau_disc))

In [ ]:
bureau.isna().mean()

SK_ID_CURR               0.00
SK_ID_BUREAU             0.00
CREDIT_ACTIVE            0.00
CREDIT_CURRENCY          0.00
DAYS_CREDIT              0.00
CREDIT_DAY_OVERDUE       0.00
DAYS_CREDIT_ENDDATE      0.06
DAYS_ENDDATE_FACT        0.37
AMT_CREDIT_MAX_OVERDUE   0.66
CNT_CREDIT_PROLONG       0.00
AMT_CREDIT_SUM           0.00
AMT_CREDIT_SUM_DEBT      0.15
AMT_CREDIT_SUM_LIMIT     0.34
AMT_CREDIT_SUM_OVERDUE   0.00
CREDIT_TYPE              0.00
DAYS_CREDIT_UPDATE       0.00
AMT_ANNUITY              0.71
dtype: float64

In [ ]:
input_missing(bureau, ls_bureau_disc, ls_bureau_cont)

SK_ID_CURR               0.00
SK_ID_BUREAU             0.00
CREDIT_ACTIVE            0.00
CREDIT_CURRENCY          0.00
DAYS_CREDIT              0.00
CREDIT_DAY_OVERDUE       0.00
DAYS_CREDIT_ENDDATE      0.00
CNT_CREDIT_PROLONG       0.00
AMT_CREDIT_SUM           0.00
AMT_CREDIT_SUM_DEBT      0.00
AMT_CREDIT_SUM_LIMIT     0.00
AMT_CREDIT_SUM_OVERDUE   0.00
CREDIT_TYPE              0.00
DAYS_CREDIT_UPDATE       0.00
dtype: float64

#### Obtener dummies discretas

In [ ]:
bureau

,SK_ID_CURR,SK_ID_BUREAU,CREDIT_ACTIVE,CREDIT_CURRENCY,DAYS_CREDIT,CREDIT_DAY_OVERDUE,DAYS_CREDIT_ENDDATE,CNT_CREDIT_PROLONG,AMT_CREDIT_SUM,AMT_CREDIT_SUM_DEBT,AMT_CREDIT_SUM_LIMIT,AMT_CREDIT_SUM_OVERDUE,CREDIT_TYPE,DAYS_CREDIT_UPDATE
0,215354,5714462,Closed,currency 1,-497,0,-153.00,0,91323.00,0.00,6229.51,0.00,Consumer credit,-131
1,215354,5714463,Active,currency 1,-208,0,1075.00,0,225000.00,171342.00,6229.51,0.00,Credit card,-20
2,215354,5714464,Active,currency 1,-203,0,528.00,0,464323.50,137085.12,6229.51,0.00,Consumer credit,-16
3,215354,5714465,Active,currency 1,-203,0,510.52,0,90000.00,137085.12,6229.51,0.00,Credit card,-16
4,215354,5714466,Active,currency 1,-629,0,1197.00,0,2700000.00,137085.12,6229.51,0.00,Consumer credit,-21
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1716423,259355,5057750,Active,currency 1,-44,0,-30.00,0,11250.00,11250.00,0.00,0.00,Microloan,-19
1716424,100044,5057754,Closed,currency 1,-2648,0,-2433.00,0,38130.84,0.00,0.00,0.00,Consumer credit,-2493
1716425,100044,5057762,Closed,currency 1,-1809,0,-1628.00,0,15570.00,137085.12,6229.51,0.00,Consumer credit,-967
1716426,246829,5057770,Closed,currency 1,-1878,0,-1513.00,0,36000.00,0.00,0.00,0.00,Consumer credit,-1508


In [ ]:
get_tabledummies(bureau, ls_bureau_disc)

,SK_ID_CURR,SK_ID_BUREAU,DAYS_CREDIT,CREDIT_DAY_OVERDUE,DAYS_CREDIT_ENDDATE,CNT_CREDIT_PROLONG,AMT_CREDIT_SUM,AMT_CREDIT_SUM_DEBT,AMT_CREDIT_SUM_LIMIT,AMT_CREDIT_SUM_OVERDUE,DAYS_CREDIT_UPDATE,CREDIT_ACTIVE_ACTIVE,CREDIT_ACTIVE_BAD DEBT,CREDIT_ACTIVE_CLOSED,CREDIT_ACTIVE_SOLD,CREDIT_CURRENCY_CURRENCY 1,CREDIT_CURRENCY_CURRENCY 2,CREDIT_CURRENCY_CURRENCY 3,CREDIT_CURRENCY_CURRENCY 4,CREDIT_TYPE_ANOTHER TYPE OF LOAN,CREDIT_TYPE_CAR LOAN,CREDIT_TYPE_CASH LOAN (NON-EARMARKED),CREDIT_TYPE_CONSUMER CREDIT,CREDIT_TYPE_CREDIT CARD,CREDIT_TYPE_INTERBANK CREDIT,CREDIT_TYPE_LOAN FOR BUSINESS DEVELOPMENT,CREDIT_TYPE_LOAN FOR PURCHASE OF SHARES (MARGIN LENDING),CREDIT_TYPE_LOAN FOR THE PURCHASE OF EQUIPMENT,CREDIT_TYPE_LOAN FOR WORKING CAPITAL REPLENISHMENT,CREDIT_TYPE_MICROLOAN,CREDIT_TYPE_MOBILE OPERATOR LOAN,CREDIT_TYPE_MORTGAGE,CREDIT_TYPE_REAL ESTATE LOAN,CREDIT_TYPE_UNKNOWN TYPE OF LOAN
0,215354,5714462,-497,0,-153.00,0,91323.00,0.00,6229.51,0.00,-131,0,0,1,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0
1,215354,5714463,-208,0,1075.00,0,225000.00,171342.00,6229.51,0.00,-20,1,0,0,0,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0
2,215354,5714464,-203,0,528.00,0,464323.50,137085.12,6229.51,0.00,-16,1,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0
3,215354,5714465,-203,0,510.52,0,90000.00,137085.12,6229.51,0.00,-16,1,0,0,0,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0
4,215354,5714466,-629,0,1197.00,0,2700000.00,137085.12,6229.51,0.00,-21,1,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1716423,259355,5057750,-44,0,-30.00,0,11250.00,11250.00,0.00,0.00,-19,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0
1716424,100044,5057754,-2648,0,-2433.00,0,38130.84,0.00,0.00,0.00,-2493,0,0,1,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0
1716425,100044,5057762,-1809,0,-1628.00,0,15570.00,137085.12,6229.51,0.00,-967,0,0,1,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0
1716426,246829,5057770,-1878,0,-1513.00,0,36000.00,0.00,0.00,0.00,-1508,0,0,1,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0


## Tabla previous application

### Imputación

In [ ]:
prev_disc = ["NAME_CONTRACT_TYPE","WEEKDAY_APPR_PROCESS_START","NAME_CASH_LOAN_PURPOSE","NAME_CONTRACT_STATUS",
        "NAME_PAYMENT_TYPE","CODE_REJECT_REASON","NAME_TYPE_SUITE","NAME_CLIENT_TYPE","NAME_GOODS_CATEGORY",
        "NAME_PORTFOLIO", "NAME_PRODUCT_TYPE", "CHANNEL_TYPE", "NAME_SELLER_INDUSTRY", "NAME_YIELD_GROUP", 
        "PRODUCT_COMBINATION"]
prev_cont = [list(prev_app.columns.drop(prev_disc))]

In [ ]:
prev_app.isna().mean()

SK_ID_PREV                    0.00
SK_ID_CURR                    0.00
NAME_CONTRACT_TYPE            0.00
AMT_ANNUITY                   0.22
AMT_APPLICATION               0.00
AMT_CREDIT                    0.00
AMT_DOWN_PAYMENT              0.54
AMT_GOODS_PRICE               0.23
WEEKDAY_APPR_PROCESS_START    0.00
HOUR_APPR_PROCESS_START       0.00
FLAG_LAST_APPL_PER_CONTRACT   0.00
NFLAG_LAST_APPL_IN_DAY        0.00
RATE_DOWN_PAYMENT             0.54
RATE_INTEREST_PRIMARY         1.00
RATE_INTEREST_PRIVILEGED      1.00
NAME_CASH_LOAN_PURPOSE        0.00
NAME_CONTRACT_STATUS          0.00
DAYS_DECISION                 0.00
NAME_PAYMENT_TYPE             0.00
CODE_REJECT_REASON            0.00
NAME_TYPE_SUITE               0.49
NAME_CLIENT_TYPE              0.00
NAME_GOODS_CATEGORY           0.00
NAME_PORTFOLIO                0.00
NAME_PRODUCT_TYPE             0.00
CHANNEL_TYPE                  0.00
SELLERPLACE_AREA              0.00
NAME_SELLER_INDUSTRY          0.00
CNT_PAYMENT         

In [ ]:
input_missing(prev_app, prev_disc, prev_cont)

SK_ID_PREV                    0.00
SK_ID_CURR                    0.00
NAME_CONTRACT_TYPE            0.00
AMT_ANNUITY                   0.22
AMT_APPLICATION               0.00
AMT_CREDIT                    0.00
AMT_GOODS_PRICE               0.23
WEEKDAY_APPR_PROCESS_START    0.00
HOUR_APPR_PROCESS_START       0.00
FLAG_LAST_APPL_PER_CONTRACT   0.00
NFLAG_LAST_APPL_IN_DAY        0.00
NAME_CASH_LOAN_PURPOSE        0.00
NAME_CONTRACT_STATUS          0.00
DAYS_DECISION                 0.00
NAME_PAYMENT_TYPE             0.00
CODE_REJECT_REASON            0.00
NAME_CLIENT_TYPE              0.00
NAME_GOODS_CATEGORY           0.00
NAME_PORTFOLIO                0.00
NAME_PRODUCT_TYPE             0.00
CHANNEL_TYPE                  0.00
SELLERPLACE_AREA              0.00
NAME_SELLER_INDUSTRY          0.00
CNT_PAYMENT                   0.22
NAME_YIELD_GROUP              0.00
PRODUCT_COMBINATION           0.00
dtype: float64

In [ ]:
prev_app = prev_app.drop(["PRODUCT_COMBINATION","HOUR_APPR_PROCESS_START","WEEKDAY_APPR_PROCESS_START","SELLERPLACE_AREA"
                        ,"NFLAG_LAST_APPL_IN_DAY"],axis=1)

In [ ]:
prev_app

,SK_ID_PREV,SK_ID_CURR,NAME_CONTRACT_TYPE,AMT_ANNUITY,AMT_APPLICATION,AMT_CREDIT,AMT_GOODS_PRICE,FLAG_LAST_APPL_PER_CONTRACT,NAME_CASH_LOAN_PURPOSE,NAME_CONTRACT_STATUS,DAYS_DECISION,NAME_PAYMENT_TYPE,CODE_REJECT_REASON,NAME_CLIENT_TYPE,NAME_GOODS_CATEGORY,NAME_PORTFOLIO,NAME_PRODUCT_TYPE,CHANNEL_TYPE,NAME_SELLER_INDUSTRY,CNT_PAYMENT,NAME_YIELD_GROUP
0,2030495,271877,Consumer loans,1730.43,17145.00,17145.00,17145.00,Y,XAP,Approved,-73,Cash through the bank,XAP,Repeater,Mobile,POS,XNA,Country-wide,Connectivity,12.00,middle
1,2802425,108129,Cash loans,25188.62,607500.00,679671.00,607500.00,Y,XNA,Approved,-164,XNA,XAP,Repeater,XNA,Cash,x-sell,Contact center,XNA,36.00,low_action
2,2523466,122040,Cash loans,15060.74,112500.00,136444.50,112500.00,Y,XNA,Approved,-301,Cash through the bank,XAP,Repeater,XNA,Cash,x-sell,Credit and cash offices,XNA,12.00,high
3,2819243,176158,Cash loans,47041.33,450000.00,470790.00,450000.00,Y,XNA,Approved,-512,Cash through the bank,XAP,Repeater,XNA,Cash,x-sell,Credit and cash offices,XNA,12.00,middle
4,1784265,202054,Cash loans,31924.40,337500.00,404055.00,337500.00,Y,Repairs,Refused,-781,Cash through the bank,HC,Repeater,XNA,Cash,walk-in,Credit and cash offices,XNA,24.00,high
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1670209,2300464,352015,Consumer loans,14704.29,267295.50,311400.00,267295.50,Y,XAP,Approved,-544,Cash through the bank,XAP,Refreshed,Furniture,POS,XNA,Stone,Furniture,30.00,low_normal
1670210,2357031,334635,Consumer loans,6622.02,87750.00,64291.50,87750.00,Y,XAP,Approved,-1694,Cash through the bank,XAP,New,Furniture,POS,XNA,Stone,Furniture,12.00,middle
1670211,2659632,249544,Consumer loans,11520.85,105237.00,102523.50,105237.00,Y,XAP,Approved,-1488,Cash through the bank,XAP,Repeater,Consumer Electronics,POS,XNA,Country-wide,Consumer electronics,10.00,low_normal
1670212,2785582,400317,Cash loans,18821.52,180000.00,191880.00,180000.00,Y,XNA,Approved,-1185,Cash through the bank,XAP,Repeater,XNA,Cash,x-sell,AP+ (Cash loan),XNA,12.00,low_normal



## Tabla pos_cash balance

### Imputación

In [ ]:
pos_disc = ["NAME_CONTRACT_STATUS"]
pos_cont = [list(poscash_b.columns.drop(pos_disc))]


In [ ]:
poscash_b.isna().mean()

SK_ID_PREV              0.00
SK_ID_CURR              0.00
MONTHS_BALANCE          0.00
CNT_INSTALMENT          0.00
CNT_INSTALMENT_FUTURE   0.00
NAME_CONTRACT_STATUS    0.00
SK_DPD                  0.00
SK_DPD_DEF              0.00
dtype: float64

In [ ]:
input_missing(poscash_b, pos_disc, pos_cont)

SK_ID_PREV              0.00
SK_ID_CURR              0.00
MONTHS_BALANCE          0.00
CNT_INSTALMENT          0.00
CNT_INSTALMENT_FUTURE   0.00
NAME_CONTRACT_STATUS    0.00
SK_DPD                  0.00
SK_DPD_DEF              0.00
dtype: float64

### Dummies discretas

In [ ]:
get_tabledummies(poscash_b, pos_disc)

,SK_ID_PREV,SK_ID_CURR,MONTHS_BALANCE,CNT_INSTALMENT,CNT_INSTALMENT_FUTURE,SK_DPD,SK_DPD_DEF,NAME_CONTRACT_STATUS_ACTIVE,NAME_CONTRACT_STATUS_AMORTIZED DEBT,NAME_CONTRACT_STATUS_APPROVED,NAME_CONTRACT_STATUS_CANCELED,NAME_CONTRACT_STATUS_COMPLETED,NAME_CONTRACT_STATUS_DEMAND,NAME_CONTRACT_STATUS_RETURNED TO THE STORE,NAME_CONTRACT_STATUS_SIGNED,NAME_CONTRACT_STATUS_XNA
0,1803195,182943,-31,48.00,45.00,0,0,1,0,0,0,0,0,0,0,0
1,1715348,367990,-33,36.00,35.00,0,0,1,0,0,0,0,0,0,0,0
2,1784872,397406,-32,12.00,9.00,0,0,1,0,0,0,0,0,0,0,0
3,1903291,269225,-35,48.00,42.00,0,0,1,0,0,0,0,0,0,0,0
4,2341044,334279,-35,36.00,35.00,0,0,1,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10001353,2448283,226558,-20,6.00,0.00,843,0,1,0,0,0,0,0,0,0,0
10001354,1717234,141565,-19,12.00,0.00,602,0,1,0,0,0,0,0,0,0,0
10001355,1283126,315695,-21,10.00,0.00,609,0,1,0,0,0,0,0,0,0,0
10001356,1082516,450255,-22,12.00,0.00,614,0,1,0,0,0,0,0,0,0,0


## Tabla bureau balance

### Imputación

In [ ]:
bur_cont = ["SK_ID_BUREAU","MONTHS_BALANCE"]
bur_disc = ["STATUS"]

In [ ]:
bureau_b.isna().mean()

SK_ID_BUREAU     0.00
MONTHS_BALANCE   0.00
STATUS           0.00
dtype: float64

In [ ]:
input_missing(bureau_b, bur_disc, bur_cont)

SK_ID_BUREAU     0.00
MONTHS_BALANCE   0.00
STATUS           0.00
dtype: float64

### Dummies discretas

In [ ]:
get_tabledummies(bureau_b, bur_disc)

,SK_ID_BUREAU,MONTHS_BALANCE,STATUS_0,STATUS_1,STATUS_2,STATUS_3,STATUS_4,STATUS_5,STATUS_C,STATUS_X
0,5715448,0,0,0,0,0,0,0,1,0
1,5715448,-1,0,0,0,0,0,0,1,0
2,5715448,-2,0,0,0,0,0,0,1,0
3,5715448,-3,0,0,0,0,0,0,1,0
4,5715448,-4,0,0,0,0,0,0,1,0
...,...,...,...,...,...,...,...,...,...,...
27299920,5041336,-47,0,0,0,0,0,0,0,1
27299921,5041336,-48,0,0,0,0,0,0,0,1
27299922,5041336,-49,0,0,0,0,0,0,0,1
27299923,5041336,-50,0,0,0,0,0,0,0,1
